In [60]:
from jupyter_dash import JupyterDash

In [80]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, ALL, MATCH
import json

import dash_bootstrap_components as dbc
import dash_html_components as html
from datetime import datetime as dt
import pandas as pd
import numpy as np
import time
import random
import plotly.express as px

import re
import locale

In [115]:
# df_articles = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_deduplicated_v4.json')
df_news = pd.read_json('/home/sid2018-6/projetInterPromo2021/Pred_Global_V0.json')
df_resume = pd.read_json('/home/sid2018-6/projetInterPromo2021/summary_v0.json')
df_articles = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_tag_clean.json')

In [120]:
df_news

,Innovant,Nouveau,Theme,art_id
0,0,0,sante,1
1,0,0,sante,2
2,1,0,france,25
3,0,0,sante,27
4,0,0,sante,28
5,0,0,sante,30
6,1,0,sante,31
7,0,0,sante,32
8,0,0,sante,34
9,0,0,sante,35


In [116]:
# dt.strptime(articles['art_extract_datetime'][0], '%d %B %Y').strftime('%d/%m/%Y')
def date_propre(extract_datetime):
    chaine = str(extract_datetime)
    mois_fr = ['janvier','février','mars','avril','mai','juin','juillet','août',
               'septembre','octobre','novembre','décembre']
    m_fr = ['janv','fév','mars','avril','mai','juin','juil','aout','sept','oct','nov','déc']
    c1 = re.compile(r'\d{2}/\d{2}/\d+')
    c2 = re.compile(r'\d+ \w+ \d{4}')
    c3 = re.compile(r'\w+ \d+, \d{4}')
    c4 = re.compile(r'\w+  \d{4}')
    c5 = re.compile(r'\d+.\d+.\d+')
    c6 = re.compile(r'\w+.\d+')
    
    
    if c1.match(chaine) is not None:
        chaine= c1.findall(chaine)[0]
        if len(chaine.split('/')[2])==2:
            return dt.strptime(chaine, '%d/%m/%y').strftime('%d/%m/%Y')
        else:
            return dt.strptime(chaine, '%d/%m/%Y').strftime('%d/%m/%Y')
        
    elif c2.match(chaine) is not None:
        try:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        except:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        
    elif c3.match(chaine) is not None:
        chaine = chaine.split()
        if len(chaine[1])==2:
            chaine[1] = '0'+str(chaine[1])
        if chaine[0].lower() in mois_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) == 4 or 'é' in chaine[0]:
            chaine[0]=chaine[0]+'.'
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) >=5:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
    
    elif c4.match(chaine) is not None:
        locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
        return(dt.strptime(chaine, '%B %Y').strftime('%d/%m/%Y'))
    
    elif c5.match(chaine) is not None:
        chaine = c5.findall(chaine)[0]
        if len(chaine.split('.')[2]) == 4:
            return(dt.strptime(chaine, '%d.%m.%Y').strftime('%d/%m/%Y'))
        else:
            return(dt.strptime(chaine, '%d.%m.%y').strftime('%d/%m/%Y'))
    
    elif c6.match(chaine) is not None:
        chaine = chaine.strip()
        if chaine.split('.')[0].lower() in m_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%b%Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%b.%Y').strftime('%d/%m/%Y'))
    
    else:
        if c2.findall(chaine)!=[]:
            return date_propre(c2.findall(chaine)[0])
        elif c1.findall(chaine)!=[]:
            return date_propre(c1.findall(chaine)[0])
        elif c3.findall(chaine)!=[]:
            return date_propre(c3.findall(chaine)[0])
        else:
            return None
    

In [119]:
df_articles = df_articles.dropna(subset=['art_title', 'art_url','art_content'])
df_articles = df_articles.reset_index(drop=True)
df_articles['art_date'] = df_articles['art_published_datetime'].apply(date_propre)
df_articles

IndexError: list index out of range

In [108]:
df_final = df_news[df_news['Innovant']==1]
df_final = df_final.merge(df_resume,on='art_id')
df_final = df_final.merge(df_articles, on= 'art_id')

In [88]:
df_final

,Innovant,Nouveau,Theme,art_id,art_summary,art_auth,art_content,art_content_html,art_extract_datetime,art_lang,art_tag,art_title,art_url,src_img,src_name,src_type,src_url,art_date
0,1,0,france,25,Quels étaient les objectifs poursuivis par le ...,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,xpath_source,http://fncdg.com/actualites/,2019-10-24
1,1,0,sante,31,"La FNCDG mène, en collaboration avec d’autres ...",None,"La FNCDG mène, en collaboration avec d’autres ...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,fr,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,http://fncdg.com/wp-content/uploads/2020/12/im...,FNCDG,xpath_source,http://fncdg.com/actualites/,2020-10-12
2,1,0,sante,36,Le décret n°2018-101 du 16 février 2018 portan...,None,Le décret n°2018-101 du 16 février 2018 portan...,"<p style=""text-align: justify;"">Le décret n°20...",13 mars 2018,fr,None,Le décret sur la médiation préalable obligatoi...,http://fncdg.com/le-decret-sur-la-mediation-pr...,http://fncdg.com/wp-content/uploads/2018/03/de...,FNCDG,xpath_source,http://fncdg.com/actualites/,2018-03-13
3,1,0,sante,38,Cette nouvelle étude présente les données soci...,None,Cette nouvelle étude présente les données soci...,"<p style=""text-align: justify;"">Cette nouvelle...",20 octobre 2017,fr,None,Les données sociales nationales 2015 de la Fon...,http://fncdg.com/les-donnees-sociales-national...,http://fncdg.com/wp-content/uploads/2017/10/do...,FNCDG,xpath_source,http://fncdg.com/actualites/,2017-10-20
4,1,0,sante,39,La loi de finances pour 2017 a été publiée au ...,None,La loi de finances pour 2017 a été publiée au ...,"<p style=""text-align: justify;"">La loi de fina...",11 janvier 2017,fr,None,Loi n°2016-1917 du 29 décembre 2016 de finance...,http://fncdg.com/loi-n2016-1917-du-29-decembre...,http://fncdg.com/wp-content/uploads/2017/01/lo...,FNCDG,xpath_source,http://fncdg.com/actualites/,2017-11-01
5,1,0,sante,40,La loi n° 2019-828 du 6 août 2019 de transform...,None,La loi n° 2019-828 du 6 août 2019 de transform...,"<p style=""text-align: justify;"">La loi n° 2019...",8 août 2019,fr,None,Loi transformation Fonction Publique,http://fncdg.com/loi-transformation-fonction-p...,http://fncdg.com/wp-content/uploads/2019/08/fu...,FNCDG,xpath_source,http://fncdg.com/actualites/,2019-08-08
6,1,0,sante,50,La séance plénière du Conseil supérieur de la ...,None,La séance plénière du Conseil supérieur de la ...,"<p style=""text-align: justify;"">La séance plén...",1 mars 2019,fr,None,Plénière du CSFPT du 27 février 2019,http://fncdg.com/pleniere-du-csfpt-du-27-fevri...,http://fncdg.com/wp-content/uploads/2019/03/pl...,FNCDG,xpath_source,http://fncdg.com/actualites/,2019-01-03
7,1,0,sante,54,La loi n°2017-86 du 27 janvier 2017 relative à...,None,La loi n°2017-86 du 27 janvier 2017 relative à...,"<p style=""text-align: justify;"">La loi n°2017-...",23 février 2017,fr,None,Promulgation de la loi Égalité et Citoyenneté,http://fncdg.com/promulgation-de-la-loi-egalit...,http://fncdg.com/wp-content/uploads/2017/02/lo...,FNCDG,xpath_source,http://fncdg.com/actualites/,2017-02-23
8,1,0,sante,64,"Michel HIRIART, Président de la Fédération, a ...",None,"Michel HIRIART, Président de la Fédération, a ...","<p style=""text-align: justify;"">Michel HIRIART...",11 décembre 2019,fr,None,Rencontres du President de la Fédération avec ...,http://fncdg.com/rencontres-du-president-de-la...,http://fncdg.com/wp-content/uploads/2019/12/po...,FNCDG,xpath_source,http://fncdg.com/actualites/,2019-11-12
9,1,0,sante,65,Le décret n°2014-513 du 20 mai 2014 a créé un ...,None,Le décret n°2014-513 du 20 mai 2014 a créé un ...,"<p style=""text-align: justify;"">Le décret n°20...",11 janvier 

In [118]:
df_marine

,art_auth,art_content,art_content_html,art_id,art_img,art_lang,art_published_datetime,art_tag,art_title,art_url,src_name,src_type,src_url
0,None,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",1,http://fncdg.com/wp-content/uploads/2020/09/im...,fr,22 septembre 2020,None,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/
1,None,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",2,http://fncdg.com/wp-content/uploads/2020/03/co...,fr,17 mars 2020,None,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/
2,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",25,http://fncdg.com/wp-content/uploads/2019/10/in...,fr,24 octobre 2019,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,FNCDG,xpath_source,http://fncdg.com/actualites/
3,None,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",27,http://fncdg.com/wp-content/uploads/2017/05/pu...,fr,31 mai 2017,None,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,FNCDG,xpath_source,http://fncdg.com/actualites/
4,None,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",28,http://fncdg.com/wp-content/uploads/2017/03/Sa...,fr,13 mars 2017,None,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,FNCDG,xpath_source,http://fncdg.com/actualites/
5,None,L’une des innovations de la loi n°2019-828 du ...,"<p style=""text-align: justify;"">L’une des inno...",30,http://fncdg.com/wp-content/uploads/2020/10/LG...,fr,22 octobre 2020,None,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,FNCDG,xpath_source,http://fncdg.com/actualites/
6,None,"La FNCDG mène, en collaboration avec d’autres ...","<p style=""text-align: justify;"">La FNCDG mène,...",31,http://fncdg.com/wp-content/uploads/2020/12/im...,fr,10 décembre 2020,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,FNCDG,xpath_source,http://fncdg.com/actualites/
7,None,"Créé pour et par les décideurs territoriaux, É...","<p style=""text-align: justify;"">Créé pour et p...",32,http://fncdg.com/wp-content/uploads/2017/02/re...,fr,24 février 2017,None,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,FNCDG,xpath_source,http://fncdg.com/actualites/
8,None,Les décrets n°2017-397 et n°2017-318 du 24 mar...,"<p style=""text-align: justify;"">Les décrets n°...",34,http://fncdg.com/wp-content/uploads/2017/04/po...,fr,5 avril 2017,None,Le cadre d’emplois des agents de police munici...,http://fncdg.com/le-cadre-demplois-des-agents-...,FNCDG,xpath_source,http://fncdg.com/actualites/
9,None,Une candidate à un examen professionnel organi...,"<p style=""text-align: justify;"">Une candidate ...",35,http://fncdg.com/wp-content/uploads/2017/07/Co...,fr,6 juillet 2017,None,Le Conseil d’Etat confirme la souveraineté des...,http://fncdg.com/le-conseil-detat-confirme-la-...,FNCDG,xpath_source,http://fncdg.com/actualites/


In [87]:
df_final['art_date']=pd.to_datetime(df_final['art_date'])

In [12]:
def nettoyage(themes_extrait):
    #Création de la liste de sortie
    themes_nettoyes=[]
    #Pour chaque champ, on enlève les caractères 'parasite', on ajoute le résultat dans la liste de sortie
    for i in range(0,len(themes_extrait)):
        if ',' in themes_extrait[i]:
            for j in themes_extrait[i].split(','):
                themes_nettoyes.append(j.strip(" '\"[]#"))
        else:
            themes_nettoyes.append(themes_extrait[i].strip(" '\"[]#"))
    #On enlève les doublons
    themes_nettoyes=list(np.unique(themes_nettoyes))
    themes_nettoyes.remove('')
    return themes_nettoyes

def traitement_themes():
    #Import des la liste des thèmes
    themes=list(df_final['art_tag'].dropna())
    #On enlève les doublons
#     themes_uniques=list(np.unique(themes))
#     #Appel de la fonction extraction
#     themes_extrait=extraction(themes_uniques)
    #Appel de la fonction nettoyage
    themes_finaux=nettoyage(themes)
    #Retourne les thèmes
    return themes_finaux

def Importation(name: str):
    '''Documentation
    Parameters: name refert to the file name
    output: DataFrame
    '''
    return pd.read_excel(name)

#Function to create language options for the dropdown
def get_lang_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_lang: 'lang' field with no errors and duplicates
        dict: dictionary with the languages found in the database
        
    output:
        lang_options: list of dictionaries for dropdowns
    """
    list_lang=articles['art_lang'].dropna().unique()
    #Declaration of lang_options. It needs to be declared in order to use the append method.
    lang_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_lang)):
        dict={'label':list_lang[n],'value':list_lang[n]}
        lang_options.append(dict)
    return lang_options

#Fonction qui estime le temps de lecture
def count_words(string:str):
    """ 
    Parameters:
        string: We take the words of the article as input
    Attributes:
        string1: it allows to remove the beginning and end space
        count: we initialize count to 1, count will be our word counter
        second: we know that we read 275 words in 1 minute, so we calculate the number of seconds it would take to read the article.
        minute: we convert the number of seconds into minutes
        second2: we make the difference between a minute and the whole part of a minute, which is multiplied by 60. This gives us the remaining seconds
    Out:
        out1: maximum estimated reading time in minutes

    """
    string1=str(string).strip()
    count=1
    for i in string1:
        # a for is used to count the number of words in the article
        if i==" ":
            count+=1
    second2=(count*60)/275
    minute=second/60
    second2=(minute-math.floor(minute))*60
    if second2>0:
        return ("temps maximum de lecture estime a",math.floor(minute)+1," min ")
    else:
        return ("temps maximum de lecture estime a",math.floor(minute)," min ")

#Function to create themes options for the dropdown
def get_cat_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_cat: 'themes' field
        dict: dictionary with the languages found in the database
        
    output:
        cat_options: list of dictionaries for dropdowns
    """
    list_cat=articles['src_name'].dropna().unique()
    #Declaration of cat_options. It needs to be declared in order to use the append method.
    cat_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_cat)):
        dict={'label':list_cat[n],'value':list_cat[n]}
        cat_options.append(dict)
    return cat_options


def get_min_max_dates(articles):
    """Documentation
    
    Parameters:
        articles: we take dataframe type articles as inputs
        
    Attributes:
        date: we retrieve the column art_date where the dates are stored

    Out:
        min_date: the minimum date of the column is retrieved
        max_date: the maximum date of the column is retrieved
    """
    date = articles[['art_date']]
    min_date = min(date['art_date'])
    max_date = max(date['art_date'])
    return (min_date,max_date)

def plot_pie_chart(articles, variable : str , pie_title : str): 
    """Documentation
    
    Parameters:
    
    articles : DataFrame used to plot
    variable : Variable used to compare articles
    pie_title: Title given to the plot
    
    Out: figure 
    """
    articles[variable].fillna("Unknown", inplace = True)
    grp_variable = articles.groupby(variable)[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig = px.pie(grp_variable, values='nb_articles', names=variable, title=pie_title)
    return fig


#graphs
df_final['art_lang'].fillna("Unknown", inplace = True)
grp_languages = df_final.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
fig_lang = plot_pie_chart(df_final,'art_lang','Languages')


df2=df_final.groupby('src_name')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
fig_themes = px.histogram(df2, x="src_name", y='nb_articles')

In [97]:
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

df = df_final.dropna(subset=['art_title', 'art_url','art_content']).reset_index(drop=True)


tags = traitement_themes()
dic = {}
for i in df['art_tag']:
    if i is None :
        continue
    else :
        for j in tags:
            if j in i:
                if j in dic:
                    dic[j] += 1
                else :
                    dic[j] = 1

top_tags = sorted(dic.items(),key = lambda t:t[1])[-12:-2]

tags_index = 11

# Search Bar

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(id = "username",type="search", placeholder="Search",debounce=True)),
        dbc.Col(
            dbc.Button("Search", id = "submit-button",color="primary", className="ml-2",href="/"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Nav Bar

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Berger-levrault", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://www.berger-levrault.com/fr/",
        ),
        dbc.NavLink("Article", active=True, href="/"),
        dbc.NavLink("Statistiques", href="/stats"),
        dbc.NavLink("Stories",href="/stories"),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
    fixed = "top",
    sticky = "top",
)


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "margin-left": "2rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

#Side bar

sidebar = html.Div(
    [
        html.H2("Home"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Number of article par page'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10,
                    min = 1,
                    max = len(df['art_title']),
                    )])),
            dbc.Col(html.Div([html.Label('Language'),
                              dcc.Dropdown(id='lang_filt',
                    options=get_lang_options(df),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Themes'),
                              dcc.Dropdown(id='cat_filt',
                    options=get_cat_options(df),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date= '01/01/2010',
                    end_date= '01/01/2020'
                    # end_date_placeholder_text='Select a date!'
                    )]), width='auto'),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)


#Nav bar for pagging

nb_pages = 10

pages = dbc.NavbarSimple(
    children=[
        dbc.Col(html.Div(html.H5("Page", style = {"color":"white"}))),
        dbc.Col(html.Div([dcc.Input(
                    id = 'num_page',
                    type= "number",
                    value = 1,
                    min = 1,
                    style = {"border-radius": "8px", "color": "black", "border": "2px solid #e7e7e7",
                            "width" : "30%",
                            "background-color": "#eee"}
                    )])),
    ],
    color="dark",
)

#Stories

def render_stories():
    list_tags = []
    for i in range(len(top_tags)-1,-1,-1):
        list_tags.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([
                    dbc.CardBody(
                        [
                            html.H5(top_tags[i][0], className="card-title"),
                            html.H6("Number of article : " + str(top_tags[i][1]), className="card-subtitle"),
                            dbc.Button("Show this tags",id = {'type': 'tags-button','index': i}),   
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    
    
    return list_tags 

def affichage_card(liste_art,df,l_index=[]):   
    """
    Input:
        liste_art : list of article index in int
        l_index : list of html.Div for display
    Output:
        l_index : list of html.Div for display with the articles cards
    """
    for nb in liste_art:
        # in the index of selected articles
        l_index.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=df['src_img'][nb],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(df['art_title'][nb], className="card-title"), href='/page-'+str(df['art_id'][i])),
                            html.P(df['art_content'][i][:200])
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
    return l_index

# Content page

card = html.Div(id ='out_card')

article = html.Div(id ='out_article')

stats = html.Div(id='out_stats')

stories = html.Div(render_stories())


app.layout = html.Div(
        [
            navbar,
            html.Hr(),
            dcc.Location(id='url', refresh=False),
            html.Div(id='page-content'),
            pages
        ]
)

index_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(card),
                                dbc.Col(stories,width = "30%")
                            ],
                            no_gutters=True,
                        ),

stats_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(stats),
                                dbc.Col(
                                            dcc.Graph(figure = fig_lang,id='graph_lang'),
                                            width={'size':3}
                                        ),
                                        dbc.Col(
                                            dcc.Graph(figure = fig_themes,id='graph_theme'),
                                            width={'size':5}
                                        )
                            ],
                            no_gutters=True,
                        ),

article_page_layout = dbc.Col(article)

stories_page_layout = dbc.Col(stories)


# Callback

# add callback for toggling the collapse on small screens
@app.callback(
    dash.dependencies.Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


@app.callback(
    dash.dependencies.Output('out_card', 'children'),
    dash.dependencies.Output('num_page','max'),
    [Input('number', 'value')],
    [Input('num_page','value')],
    [Input({'type': 'tags-button','index': ALL},'n_clicks')],
    [Input('submit-button', 'n_clicks')],
    [Input('username', 'value')],
    [State('username', 'value')],
)

def update_card_div(number,num_page,n,clicks,event,input_value):
    
    global tags_index

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
#     filtered_df_news = df[df['Nouveau']==1]
    filtered_df = df
    
    if "index" in changed_id:
        tags_index = int(changed_id[9])
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
        
    if tags_index != 11:
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
    
    filtered_df = filtered_df.reset_index(drop=True)
    
    element = []
    if clicks is not None or event is not None:
        for nb in range(len(filtered_df)):
            t = input_value
            if t.lower() in filtered_df['art_title'][nb].lower() or t.lower() in filtered_df['art_content'][nb].lower() or t.lower() in filtered_df['art_tag'][nb].lower():
                element.append(filtered_df['art_id'][nb])
    
    if element != []:
        filtered_df = filtered_df[filtered_df['art_id'].isin(element)]
 
    filtered_df = filtered_df.reset_index(drop=True)
    
    length_df = len(filtered_df['art_title'])
    
    if num_page*number > length_df:
        end = length_df
        filtered_df = filtered_df[((num_page*number)-number):length_df]
    else :
        filtered_df = filtered_df[((num_page*number)-number):((num_page*number))]
        end = num_page*number
    
    cards = []
    
    for i in range(((num_page*number)-number),end):
        cards.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=filtered_df['src_img'][i],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(filtered_df['art_title'][i], className="card-title"), href='/page-'+str(filtered_df['art_id'][i])),
                            html.P(filtered_df['art_content'][i][:200])
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    return cards,(length_df//number)+1

#Content article

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
    
    index = int(pathname.split("-")[1])
    
    df_article = df[df['art_id'] == index]
    df_article = df_article.reset_index()
    
    print(df_article['art_title'][0])

    article = [html.Div([
            dbc.Card([
                dbc.CardImg(src=df_article['src_img'][0],alt="Can't display this image",top=True),
                dbc.CardBody(
                    [
                        dbc.CardLink(html.H5(df_article['art_title'][0], className="card-title"), href=df_article['art_url'][0]),
                        html.P(df_article['art_content'][0])
                    ]
                )],
                className="w-100 mb-3"
            ),
        ])
    ]
    
    print("test")

    return article


# Callback for update data in graphs
@app.callback(
    Output('graph_lang', 'figure'),
    Output('graph_theme', 'figure')
    ,
    [Input('lang_filt', 'value'),
    Input('cat_filt','value'),
    Input('date-picker-range','start_date'),
    Input('date-picker-range','end_date')
    ]
    )
def update_figure(language,theme,start,end):
    
    filtered_df_2 = df
    
    filtered_df_2['art_lang'].fillna("Unknown", inplace = True)
    # Language filter
    if language!=[]: filtered_df_2=filtered_df_2[filtered_df_2['art_lang'].isin(language)]
 
    # Filter on themes:
    if theme!=[]: filtered_df_2=filtered_df_2[filtered_df_2['src_name'].isin(theme)]
        
    #Filter on dates
#     terminal=get_min_max_dates(filtered_df_2)
#     if start!=terminal[0] or end!=terminal[1]: filtered_df_2=filtered_df_2[(filtered_df_2['art_date'] >= start) & (filtered_df_2['art_date'] <= end)]
    
    # Graph nbarticles by langue
    grp_languages=filtered_df_2.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig_lang = plot_pie_chart(filtered_df_2,'art_lang','Languages')
    fig_lang.update_layout(transition_duration=500)
    
    # Graph nbarticles by theme
    df2=filtered_df_2.groupby('src_themes')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig_themes = px.histogram(df2, x="src_themes", y='nb_articles')
    fig_themes.update_layout(transition_duration=500)
    
    return fig_lang, fig_themes

#Content    
    
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return index_page_layout
    elif pathname == "/stats":
        return stats_page_layout
    elif pathname == "/stories":
        return stories_page_layout
    else:
        return article_page_layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )




if __name__ == "__main__":
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/


In [42]:
# app.run_server(port=8050,host='127.0.0.1')